In [1]:
import pandas as pd
from lxml import etree, objectify
import os
import shutil
import in_place
import re

## Run only once
### File adjustments
The conll files originally had odd number of columns, which was a problem for pandas. The first one moves the conll file to the xml directory and the second one leaves only token number and token in the conll file.

In [24]:
for xml_root,xml_dirs,xml_files in os.walk('./data/modality_xmls/'):
    for root,dirs,files in os.walk('./data/gme-conll-tokenized/'):
        for file in files:
            if xml_root.replace('./data/modality_xmls/', '') == file.replace('.txt', ''):
                shutil.move(os.path.join(root,file), os.path.join(root,file.replace('.txt', '.conll')))
                
with in_place.InPlace(root+'/'+file) as f:
    for line in f:
        line = line.replace('\t_', '')
        f.write(line)

## Getting the tokenized version of an element

file `gme.xxxxxxx_xxx.xx.xx-xxxxx.mark.xml` contains the "entire span" as it is, and below it a list of the token numbers, like this:
```
		<!--Privateorganizationsarealsobeingencouragedtohelpfightsandstorms-->
		<mark id="other_4" xlink:href="#sTok25 #sTok26 #sTok27 #sTok28 #sTok29 #sTok30 #sTok31"/>
```
The tokens appear along their numbers in a file named `gme.xxxxxxx_xxx.xx.xx-xxxxx.tok.xml`. It looks like this:
```
		<!--Private-->
		<mark id="sTok25" xlink:href="#xpointer(string-range(//body,'',1195,7))"/>
		<!--organizations-->
		<mark id="sTok26" xlink:href="#xpointer(string-range(//body,'',1202,13))"/>
		<!--being-->
		<mark id="sTok27" xlink:href="#xpointer(string-range(//body,'',1222,5))"/>
		<!--to-->
		<mark id="sTok28" xlink:href="#xpointer(string-range(//body,'',1237,2))"/>
		<!--help-->
		<mark id="sTok29" xlink:href="#xpointer(string-range(//body,'',1239,4))"/>
		<!--fight-->
		<mark id="sTok30" xlink:href="#xpointer(string-range(//body,'',1243,5))"/>
		<!--sandstorms-->
		<mark id="sTok31" xlink:href="#xpointer(string-range(//body,'',1248,10))"/>
        ```
        
**UPDATE**

The tokens file presents the tokens not by their order in the original sentence. 
The only place where the tokens appear is in to conll file

In [14]:
entire_span_df = pd.read_csv('./data/entire_span.csv')

In [5]:
spans_with_toks = {}
for root,dirs,files in os.walk('./data/modality_xmls/'):
    for file in files:
        if file.endswith("mark.xml"):
            with open(os.path.join(root,file), 'r') as f:
                f = f.readlines()
                for x in range(len(f)):
                    s = f[x].replace("<!--", "").replace("-->", "").strip()
                    if s in spans:
                        toks = [int(t.replace("\"/>\n", "").replace("#sTok", "").replace("\"/></markList></paula>", "")) if t[0] == '#' else int(t.replace("\"/>\n", "").replace("xlink:href=\"#sTok", "").replace("\"/></markList></paula>", "")) if '#' in t else None for t in f[x+1].split(" ")]
                        toks = [t for t in toks if t]
                        file_id = re.search('(\d+\.\d+\-\d+)', file)
                        spans_with_toks[s] = {"token_ids": toks, "file_id": file_id.group(1), "tokens": []}

KeyboardInterrupt: 

In [4]:
for root,dirs,files in os.walk('./data/modality_xmls/'):
    for file in files:
        for k, v in spans_with_toks.items():
            if file.endswith("{}.conll".format(v['file_id'])):
                df = pd.read_csv(os.path.join(root,file), sep='\t', names=['id', 'token'])
                start = df[df['id'] == v['token_ids'][0]]
                end = df[df['id'] == v['token_ids'][-1]]
                v['tokens'].extend(df.iloc[start.index[0]:end.index[0]+1, :]['token'].values.tolist())

NameError: name 'spans_with_toks' is not defined

In [4]:
spans = entire_span_df['entire span'].tolist()
spans

['Privateorganizationsarealsobeingencouragedtohelpfightsandstorms',
 'Favourabletaxationpolicies',
 'Nobodyishappy',
 'urgentrepairstoinfrastructure',
 'theWorldBankof3.9billiondollarsforwhatBelgradewouldneedforthethree-yearperiod',
 'integralpartofthesuccessfulformulaoftrustforeconomicrecoveryintheregion',
 "that'sveryimportantforus",
 'mycountryisreadytorespectallinternationalobligations',
 ",saidseveraldelegationstotheBrusselsconferencetoldhimitwasnow``easierandmoremotivating''tosupplyBelgradewiththeaidit",
 'whohadhelpedhimputdowntheanimalweresenturgentlytotheHospitalforInfectiousDiseasesinMiercureaCiuc',
 'Ouragencyseriouslyneedsequipmentfordetectingdrugs',
 'nothaveenoughcommunicationsequipment',
 'Some60representativesofcivilsociety,parties,rebelgroupsandgovernmentintheDemocraticRepublicofCongo(DRC)aretotakepartinpreparatorytalksonrevivingpoliticaldialogueinthevastcountrynextmonth',
 'peoplefromallcornersofthecountryareallowedtoexpresstheirviews',
 "Thepoliticalplayersmustoverco

In [8]:
def get_segments_for_element(elem, df):
    elems = df[elem].tolist()
    dict_with_segmened_elem = {}
    for root,dirs,files in os.walk('./data/modality_xmls/'):
        for file in files:
            if file.endswith("mark.xml"):
                with open(os.path.join(root,file), 'r') as f:
                    f = f.readlines()
                    for x in range(len(f)):
                        s = f[x].replace("<!--", "").replace("-->", "").strip()
                        if s in elems:
                            toks = [int(t.replace("\"/>\n", "").replace("#sTok", "").replace("\"/></markList></paula>", "")) if t[0] == '#' else int(t.replace("\"/>\n", "").replace("xlink:href=\"#sTok", "").replace("\"/></markList></paula>", "")) if '#' in t else None for t in f[x+1].split(" ")]
                            toks = [t for t in toks if t]
                            file_id = re.search('(\d+\.\d+\-\d+)', file)
                            dict_with_segmened_elem[s] = {"token_ids": toks, "file_id": file_id.group(1), "tokens": []}
    for root,dirs,files in os.walk('./data/modality_xmls/'):
        for file in files:
            for k, v in dict_with_segmened_elem.items():
                if file.endswith("{}.conll".format(v['file_id'])):
                    df = pd.read_csv(os.path.join(root,file), sep='\t', names=['id', 'token'])
                    start = df[df['id'] == v['token_ids'][0]]
                    end = df[df['id'] == v['token_ids'][-1]]
                    v['tokens'].extend(df.iloc[start.index[0]:end.index[0]+1, :]['token'].values.tolist())
    return dict_with_segmened_elem

In [29]:
segmented_span = get_segments_for_element('bg_content', entire_span_df)

In [11]:
incorrect = []
for k, v in segmented_prej.items():
    if k != "".join(v['tokens']) and v['tokens'] == []:
        incorrect.append((k, v['token_ids'], v['file_id']))
print(len(incorrect))
print(len(segmented_prej))

0
7105


what should be taken is the entire span from the smallest to the biggest instead of 1:1

In [34]:
entire_span_df['document'] = entire_span_df['prejacent'].apply(lambda x: " ".join(segmented_prej[x]['file_id']) if x in segmented_prej else [])

In [37]:
entire_span_df.columns

Index(['Unnamed: 0', '1_id', 'entire span', 'prejacent', 'modal',
       'modal token', 'modal_lemma', 'background', 'clausal_arguments_max1',
       'clausal_arguments_max2', 'degree_indicator', 'environmental_polarity',
       'modality_type', 'modified_element', 'outscoping_quantifier',
       'predication_type', 'source', 'modal_subtype', 'src_content',
       'bg_content', 'segmented_prejacent', 'segmented_span',
       'segmented_source', 'segmented_background', 'document'],
      dtype='object')

In [39]:
cols = ['1_id', 'document','modal','modal token', 'modal_lemma', 'clausal_arguments_max1',
       'clausal_arguments_max2', 'degree_indicator', 'environmental_polarity',
       'modality_type', 'modified_element', 'outscoping_quantifier',
       'predication_type', 'modal_subtype', 'segmented_prejacent', 'segmented_span',
       'segmented_source', 'segmented_background']
entire_span_df.to_csv('./data/segmented_GME.csv', sep='\t', columns=cols)

In [42]:
entire_span_df[entire_span_df['segmented_prejacent'] == '[]']

,Unnamed: 0,1_id,entire span,prejacent,modal,modal token,modal_lemma,background,clausal_arguments_max1,clausal_arguments_max2,...,predication_type,source,modal_subtype,src_content,bg_content,segmented_prejacent,segmented_span,segmented_source,segmented_background,document
